In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import FloatSlider, interact, interactive, fixed, interact_manual
from IPython.display import display
import os
import warnings

pd.get_option("display.max_columns", None)
pd.get_option("display.max_rows", None)
warnings.filterwarnings("ignore", message="Glyph 9 missing from current font.")

In [3]:
layout = widgets.Layout(width='auto', height='40px')

In [4]:
p = widgets.FloatSlider(
    value=0.05,
    min=0,
    max=0.2,
    step=0.00001,
    description='P',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
    layout=layout
)

In [5]:
q = widgets.FloatSlider(
    value=0.3,
    min=0,
    max=1,
    step=0.0001,
    description='Q',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.4f',
    layout=layout
)

In [6]:
m = widgets.FloatSlider(
    value=20,
    min=1,
    max=10000,
    step=1,
    description='M',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
    layout=layout
)

In [21]:
def run_simulation(p,q,m):
    '''
    Run simulation and return historical observations for 2015-2021 and Bass model projectiosn for 2022-2030
    '''
    s = pd.Series(0, range(2015,2031))
    S = pd.Series(0, range(2015,2031))
    f = pd.Series(0, range(2015,2031))
    F = pd.Series(0, range(2015,2031))

    s.loc[2015] = p*m
    S.loc[2015] = 0
    f.loc[2015] = p
    F.loc[2015] = 0
        
    for i in range(2016,2031):
        S.loc[i] = S.loc[i-1] + s.loc[i-1]
        F.loc[i] = F.loc[i-1] + f.loc[i-1]
        f.loc[i] = (p + (q * F.loc[i])) * (1 - F.loc[i])
        s.loc[i] = f.loc[i] * m
    f*=100
    F*=100
    
    fig, axes = plt.subplots(2, 2, figsize=(12,8))
    s.plot(ax=axes[0,0], xticks=(np.arange(2014, 2031, 2)), ylabel='Units', title='Sales', legend=False)
    S.plot(ax=axes[0,1], xticks=(np.arange(2014, 2031, 2)), ylabel='Units (cumulative)',
           title='Cumulative Sales', legend=False)
    f.plot(ax=axes[1,0], xticks=(np.arange(2014, 2031, 2)), ylabel='Percentage points', title='Sales PDF', legend=False)
    F.plot(ax=axes[1,1], xticks=(np.arange(2014, 2031, 2)), ylabel='%', title='Sales PDF', legend=False)
    axes[1,1].set_ylim(0,100)
    axes[1,0].set_ylim(0,35)
#    axes[0,1].set_ylim(0,((m//100)+1)*100)
#    axes[0,0].set_ylim(0,(((m*0.35)//5)+1)*5)

In [22]:
ui = widgets.VBox([p,q,m])
out = widgets.interactive_output(run_simulation, {'p': p, 'q': q, 'm': m})

display(ui, out)

Output()